# 8. 搭建一个待评估的端到端问答系统

## 2. 端到端实现问答系统

In [2]:
import openai
import utils_zh
from tool import get_completion_from_messages

'''
注意：限于模型对中文理解能力较弱，中文 Prompt 可能会随机出现不成功，可以多次运行；也非常欢迎同学探究更稳定的中文 Prompt
'''
def process_user_message_ch(user_input, all_messages, debug=True):
  """
  对用户信息进行预处理
  
  参数:
  user_input : 用户输入
  all_messages : 历史信息
  debug : 是否开启 DEBUG 模式,默认开启
  """
  
  # 分隔符
  delimiter = "```"
  
  # 第一步，使用OpenAI的Moderation API 检查用户输入是否合规
  # response = openai.moderations.create(input=user_input)
  # moderations_output = response.results[0]
  
  # # 经过 Moderation API 检查该输入不合规
  # if moderations_output["flagged"]:
  #   print("第一步：输入被 Moderation 拒绝")
  #   return "抱歉，您的请求不合规"
  
  # # 如果开启了 DEBUG 模式，打印实时进度
  if debug: print("第一步：输入通过 Moderation 检查")
  
  # 第二步：抽取出商品和对应的目录，类似于之前课程中的方法，做了一个封装
  category_and_product_response = utils_zh.find_category_and_product(user_input, utils_zh.get_products_and_category())
  # 将抽取出来的字符串转化为列表
  category_and_product_list = utils_zh.read_string_to_list(category_and_product_response)
  
  if debug: print("第二步：抽取出商品列表")
  
  # 第三步：查找商品对应信息
  product_information = utils_zh.generate_output_string(category_and_product_list)
  if debug: print("第三步：查找抽取出的商品信息")
  
  # 第四步：根据信息生成回答
  system_message = f"""
    您是一家大型电子商店的客户服务助理。\
    请以友好和乐于助人的语气回答问题，并提供简洁明了的答案。\
    请确保向用户提出相关的后续问题。
  """
  # 插入message
  messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
    {'role': 'assistant', 'content': f"相关商品信息：\n{product_information}"},
  ]
  # 获取 GPT3.5 的回答
  # 通过附加 all_messages 实现多轮对话
  final_response = get_completion_from_messages(all_messages + messages)
  if debug: print("第四步：生成用户回答")
  
  # 将该轮信息加入到历史信息中
  all_messages = all_messages + messages[1:]
  
  # 第五步：基于Moderation API检查输出是否合规
  # response = openai.Moderation.create(input=final_response)
  # moderation_output = response["results"][0]

  # # 输出不合规
  # if moderation_output["flagged"]:
  #   if debug: print("第五步：输出被 Moderation 拒绝")
  #   return "抱歉，我们不能提供该信息"

  # if debug: print("第五步：输出经过 Moderation 检查")
  
  # 第六步：模型检查是否很好地回答了用户问题
  user_message = f"""
    用户信息: {delimiter}{user_input}{delimiter}
    代理回复: {delimiter}{final_response}{delimiter}

    回复是否足够回答问题
    如果足够，回答 Y
    如果不足够，回答 N
    仅回答上述字母即可
  """
  
  messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': user_message}
  ]
  # 要求模型评估回答
  evaluation_response = get_completion_from_messages(messages)
  if debug: print("第六步：模型评估该回答")
  
  # 第七步：如果评估为 Y，输出回答；如果评估为 N，反馈将由人工修正答案
  if "Y" in evaluation_response: # 使用 in 来避免模型可能生成 Yes
    if debug: print("第七步：模型赞同了该回答.")
    return final_response, all_messages
  else:
    if debug: print("第七步：模型不赞成该回答.")
    neg_str = "很抱歉，我无法提供您所需的信息。我将为您转接到一位人工客服代表以获取进一步帮助。"
    return neg_str, all_messages
  
user_input = "请告诉我关于 smartx pro phone 和 the fotosnap camera 的信息。另外，请告诉我关于你们的tvs的情况。"
response, _ = process_user_message_ch(user_input, [])
print(response)
  
  

第一步：输入通过 Moderation 检查
第二步：抽取出商品列表
第三步：查找抽取出的商品信息
第四步：生成用户回答
第六步：模型评估该回答
第七步：模型赞同了该回答.
您好！很高兴为您提供产品信息。

**关于 SmartX Pro Phone：**
这是一款高性能智能手机，配备6.1英寸显示屏、128GB存储和12MP双摄像头，支持5G网络。目前售价899.99美元，享有1年保修，用户评分4.6/5。

**关于 FotoSnap Camera：**
这款数码单反相机拥有24.2MP传感器，支持1080p视频拍摄和可更换镜头，配备3英寸LCD屏幕。售价599.99美元，享1年保修，用户评分4.7/5。

**关于电视产品：**
我们提供多款CineView品牌电视：
- **4K TV（55英寸）**：支持HDR和智能功能，售价599.99美元
- **8K TV（65英寸）**：顶级画质，售价2999.99美元  
- **OLED TV（55英寸）**：色彩表现优异，售价1499.99美元
全部享有2年保修，评分均超过4.7/5。

需要了解某款产品的详细规格，或者想知道目前的促销活动吗？ 😊


## 2. 持续收集用户的助手消息

In [10]:
def collect_messages_ch(debug=True):
  """
  用于收集用户的输入并生成助手的回答

  参数：
  debug: 用于觉得是否开启调试模式
  """
  
  user_input = inp.value_input
  if debug: print(f"User Input = {user_input}")
  if user_input == "":
    return 
  inp.value = ''
  global context
  # 调用 process_user_message 函数
  response, context = process_user_message_ch(user_input, context, debug=False)
  context.append({'role': 'assistant', 'content': f"{response}"})
  panels.append(
    pn.Row('User:', pn.pane.Markdown(user_input, width=600))
  )
  panels.append(
    pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'}))
  )
  
  return pn.Column(*panels) 

In [11]:
import panel as pn 
pn.extension()

panels = [] # collect display

# 系统信息
context = [{'role': 'system', 'content': "you are Service Assistant"}]

inp = pn.widgets.TextInput(placeholder="Enter text here……")
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages_ch, button_conversation)

dashboard = pn.Column(
  inp,
  pn.Row(button_conversation),
  pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'06bf3c52-85fd-4bae-9763-b4395759caf6': {'version…